# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: NIRSpec fitting of the CO band-heads to get stellar velocities

Michael Reefe

This example notebook provides a tutorial on how to run LOKI on a NIRSpec IFU cube, zooming in on the CO band heads and fitting high resolution stellar templates to obtain precise stellar velocity measurements.

In [1]:
using Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
Pkg.precompile()
using Loki
using Unitful 

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


Now we want to load in our data. For this example, we'll be using the data for M87, which are located in the same folder as this notebook. The JWST reduced data does not include a redshift, so we must provide the redshift ourselves.  We can use the `from_fits` function to load in the JWST-formatted FITS files, along with the redshift.

In [2]:
# The redshift of the target object: M87
z = 0.004283
# The semicolon at the end suppresses printing the output Observation object, which is long and not very enlightening
obs = from_fits(["jw02228-o001_t001_nirspec_g235h-f170lp_s3d.fits.gz"], z);

[ Info: Initializing DataCube struct from jw02228-o001_t001_nirspec_g235h-f170lp_s3d.fits.gz


Next, we create some variables that we will use later. We will be fitting data from the G235H grating and the F170LP filter, and we can take the `name` property from the Observation object we just loaded in to get the name of the target. Here, `run_name` is just a unique identifier that we will use for this run.

In [3]:
channel = :G235H_F170LP
nm = replace(obs.name, " " => "_") 
run_name = "$(nm)_$(channel)_stel_vel"

"M-87_G235H_F170LP_stel_vel"

Before fitting, we want to do some pre-processing on the data. We want to convert the data to the rest-frame, mast out / interpolate any bad pixels, and replace the JWST pipeline-generated errors with some more realistic ones.  All of this is achieved in the next block of code.

In [4]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)

    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, [channel]);
end

[ Info: Initializing DataCube struct from M-87.channelG235H_F170LP.rest_frame.fits


Observation(Dict{Any, DataCube}(:G235H_F170LP => DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[1.6531176636622043 μm, 1.6535119748160731 μm, 1.6539063800233114 μm, 1.6543008793063532 μm, 1.6546954726876375 μm, 1.65509016018961 μm, 1.6554849418347202 μm, 1.6558798176454241 μm, 1.6562747876441823 μm, 1.6566698518534615 μm  …  3.1497442460318417 μm, 3.15049554118484 μm, 3.1512470155410885 μm, 3.1519986691433317 μm, 3.152750502034325 μm, 3.153502514256833 μm, 3.1542547058536314 μm, 3.1550070768675047 μm, 3.1557596273412494 μm, 3.1565123573176703 μm], Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}[NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ … NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹; NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ …

We next create an aperture to define the region of interest that we would like to fit. We can do this with the `make_aperture` function. We can customize the aperture's shape, centroid, radius, etc.

In [5]:
# - The first argument is the data cube
# - The second argument is the aperture shape, which may be one of: (Circular, Rectangular, Elliptical)
# - Next are the right ascension in sexagesimal hours and the declination in sexagesimal degrees
# - The next arguments depend on the aperture shape:
#    - For circles, it is the radius in arcseconds
#    - For rectangles, it is the width in arcseconds, height in arcseconds, and rotation angle in degrees
#    - For ellipses, it is the semimajor axis in arcseconds, semiminor axis in arcseconds, and rotation angle in degrees
# - The auto_centroid argument, if true, will adjust the aperture centroid to the closest peak in brightness
# - The scale_psf argument, if true, will create a series of apertures (as a function of wavelength) with increasing radii 
#   that scale at the same rate as the PSF
ap = make_aperture(obs.channels[channel], :Circular, "12:30:49.4114", "+12:23:28.150", 0.5, auto_centroid=true)

[ Info: Creating a circular aperture at 12:30:49.4114, +12:23:28.150
[ Info: Aperture centroid adjusted to -11h29m10.591302312552244s, 12d23m28.160422074724494s


11×11 Photometry.Aperture.CircularAperture{Float64} with indices 21:31×18:28:
 0.0         0.0       0.0       …  0.0       0.0        0.0
 0.0         0.0       0.293777     0.529184  0.0123099  0.0
 0.0         0.276542  0.985896     1.0       0.590527   0.0
 0.00957273  0.864973  1.0          1.0       0.997951   0.204684
 0.210345    1.0       1.0          1.0       1.0        0.538435
 0.327063    1.0       1.0       …  1.0       1.0        0.655153
 0.240608    1.0       1.0          1.0       1.0        0.568698
 0.0251184   0.914244  1.0          1.0       1.0        0.267452
 0.0         0.374704  0.999729     1.0       0.701374   0.00114812
 0.0         0.0       0.427376     0.676617  0.0468477  0.0
 0.0         0.0       0.0       …  0.0       0.0        0.0

Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data. Note specifically for NIRSpec data, we want to enable the option "nirspec_mask_chip_gaps" (see the other notebooks for an explanation of why this is necessary).

For the stellar templates, I've chosen the `stellar_template_type="stars"`, which uses individual star templates rather than stellar population templates (which is the default, `stellar_template_type="ssp"`).  The "stars" option is the most flexible in terms of fitting different continuum shapes, and it is the highest resolution, but it only covers the red-optical and NIR.  If you find that you have a spectrum where the built-in stellar templates are insufficient, first try playing with the "stars" options in the options.toml file.  Here you can adjust the limits on effective temperature, gravity, metallicity, and alpha enhancement which are included in the templates.  But be careful not to include too many templates at once, otherwise the fitting will *really* slow down.  

Note that for this example I'm also masking out everything except the region right around the CO band-heads. This is to ensure that I'm getting the most accurate velocities possible.

In [14]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
cube_fitter = CubeFitter(
    obs.channels[channel], 
    obs.z, 
    run_name; 
    nirspec_mask_chip_gaps=true,
    parallel=false, 
    plot_spaxels=:both, 
    plot_maps=true, 
    save_fits=true,
    silicate_absorption="d+",
    extinction_screen=true, 
    use_pah_templates=true,
    fit_sil_emission=false, 
    fit_stellar_continuum=true, 
    stellar_template_type="stars",    # try changing between "stars", "ssp", and "custom" and see how the results differ!
    save_full_model=true, 
    map_snr_thresh=3., 
    user_mask=[(1.0u"μm", 2.2u"μm"), (2.4u"μm", 4.0u"μm")],
    plot_range=[(2.2u"μm", 2.4u"μm")],
    # Note: what I'm doing here is just setting some arbitrary polynomial templates for the "custom" option.
    # This is just for demonstrational purposes and shouldn't really be used in any practical circumstances.
    custom_stellar_template_wave=obs.channels[channel].λ,
    custom_stellar_templates=[
        ones(length(obs.channels[channel].λ)) ustrip.(obs.channels[channel].λ) ustrip.(obs.channels[channel].λ.^2)
    ] .* unit(obs.channels[channel].I[1])
)

println(size(cube_fitter.ssps.templates))

# Call the fit_cube! function on the cube_fitter object, using the aperture we defined.
fit_cube!(cube_fitter, ap)

[ Info: Preparing output directories
┌ Warning: Setting ssp_regularize=0.0 since stellar_template_type is not "ssp"
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/cubefit.jl:320
┌ Warning: There are no PAH features within the input wavelength range of the spectrum. The use_pah_templates option will be disabled.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/create_params.jl:506
[ Info: Resampling wavelengths...


Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.87 ms/it)


[ Info: Applying instrumental broadening...


Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.83 ms/it)


[ Info: Normalizing templates...
(2880, 259)
┌ Info: 
│ 
│ #############################################################################
│ ######## BEGINNING FULL CUBE FITTING ROUTINE FOR M-87_G235H_F170LP_stel_vel ########
│ #############################################################################
│ 
│ ------------------------
│ Worker Processes:     1
│ Threads per process:  1
└ ------------------------
[ Info: ===> Preparing output data structures... <===
[ Info: Performing aperture photometry to get an integrated spectrum...
[ Info: ===> Beginning integrated spectrum fitting... <===
[ Info: ===> Generating parameter maps and model cubes... <===
[ Info: ===> Writing FITS outputs... <===
┌ Info: 
│ 
│ #############################################################################
│ ################################### Done!! ##################################
└ #############################################################################


(CubeFitter{Float64, Int64, Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(km, s⁻¹), 𝐋 𝐓⁻¹, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}(DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[1.6531176636622043 μm, 1.6535119748160731 μm, 1.6539063800233114 μm, 1.6543008793063532 μm, 1.6546954726876375 μm, 1.65509016018961 μm, 1.6554849418347202 μm, 1.6558798176454241 μm, 1.6562747876441823 μm, 1.6566698518534615 μm  …  3.1497442460318417 μm, 3.15049554118484 μm, 3.1512470155410885 μm, 3.1519986691433317 μm, 3.152750502034325 μm, 3.153502514256833 μm, 3.1542547058536314 μm, 3.1550070768675047 μm, 3.1557596273412494 μm, 3.1565123573176703 μm], Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹,

If you are still finding unsatisfactory fitting results with the "stars" method and adjusting the paremter limits, you can use your own custom-made stellar templates with the "custom" option.  Just set `stellar_template_type="custom"` and then input the wavelength vector with the `custom_stellar_template_wave` argument, and the templates themselves with the `custom_stellar_templates` argument.  For example:
```julia
# make sure your inputs have units!
wunit = u"μm"
funit = u"erg/s/cm^2/Hz/sr"
# just making up some dummy data...
wavelengths = [1., 2., 3., 4., 5.] .* wunit     # your wavelength vector
templates = zeros(typeof(1.0*funit), (length(wavelength), 3))  # your templates:
templates[:,1] .= [1., 1., 1., 1., 1.]   .* funit              # 1st axis (->) = wavelength
templates[:,2] .= [1., 1.5, 2., 2.5, 3.] .* funit              # 2nd axis (\/) = each individual template
templates[:,3] .= [5., 4., 3., 2., 1.]   .* funit              # ...

# it's okay if templates are normalized, but the units are still needed to tell
# if they are per-unit-wavelength or per-unit-frequency (they will be renormalized
# afterwards so there is no need to worry about scaling)

cube_fitter = CubeFitter(
    obs.channels[channel],
    obs.z,
    run_name;
    nirspec_mask_chip_gaps=true,
    stellar_template_type="custom",
    custom_stellar_template_wave=wavelengths,
    custom_stellar_templates=templates,
    # ... other cubefitter arguments ...
)
```

And the results can be found in the "output_[run_name]" directory, just like the other examples!
Zooming in on the CO band-heads, around the region that we fit, the continuum is reproduced very well by the models:

![](./M87.CObandheads.png)

The gray band(s) shows a masked out part of the spectrum -- this is due to NIRSpec's chip gaps.  Of course, the models are not *perfect*.  For example there appear to be some narrow absorption lines that are not captured.  We could try to improve the fit even further by expanding the effective temperature, gravity, and metallicity parameter ranges in the options.toml file, to include more stellar templates.  But there is a tradeoff between flexibility and runtime.  The more templates you include, the slower the fitting will take.